In [ ]:
import sys
sys.path.append("..")
from SE3.utils.utils import SE3_hat, is_SE3
from SO3.utils.curve_utils import log_map
import numpy as np

In [ ]:
def rk4_step(func, t, y, dt):
    k1 = func(t, y)
    k2 = func(t + dt/2, y + dt/2 * k1)
    k3 = func(t + dt/2, y + dt/2 * k2)
    k4 = func(t + dt, y + dt * k3)
    return y + dt/6 * (k1 + 2*k2 + 2*k3 + k4)

def solve_ivp_rk4(func, y0, t):
    y = np.empty((len(t), len(y0)))
    y[0] = y0
    for i in range(1, len(t)):
        y[i] = rk4_step(func, t[i-1], y[i-1], t[i] - t[i-1])
    return y

In [ ]:
def create_g_dot(xi):
    def g_dot(t, g):
        g_matrix = np.reshape(g, (4, 4))
        xi_matrix = SE3_hat(xi(t))
        return np.ravel(g_matrix @ xi_matrix)
    return g_dot




xi_1 = lambda t: np.array([2 * np.sin(3*t) * np.exp(t), 4 * np.cos(3 * t), 3 * t* np.sin(t) * np.cos(t), t, np.cos(t), np.sin(3*t)]) 
xi_2 = lambda t: np.array([t * 3, 4 * np.sin(10 * t), 3 * t* np.sin(t) * np.cos(t), np.sin(2*t), np.cos(3*t), np.exp(t)]) 
xi_3 = lambda t: np.array([4 * t ** 2, 5 * np.sin(4 * t) * np.sin(6 * t), 3 * t * np.cos(t), np.cos(t)*np.cos(3*t), np.sin(4 * t), np.cos(2 * t)]) 

t4 = lambda t: np.array([t**2, np.sin(t**2), np.exp(-t)])
t5 = lambda t: np.array([np.cos(2*t), np.log(t + 1), t])
t6 = lambda t: np.array([np.sin(t) * np.cos(t), t**3, np.sin(t**3)])
t7 = lambda t: np.array([np.cos(5*t), np.exp(t/2), t * np.sin(t)])
t8 = lambda t: np.array([np.sin(2*t), t * np.exp(-t), np.cos(t**2)])
t9 = lambda t: np.array([t**2 * np.cos(t), np.sin(t**2), -1/2])

xi_4 = lambda t: np.concatenate([np.array([3 * np.sin(2 * t) * np.exp(-t), 5 * np.cos(2 * t), 4 * t * np.sin(t) * np.cos(t)]), t4(t)])
xi_5 = lambda t: np.concatenate([np.array([t ** 3, 5 * np.sin(5 * t), 2 * t * np.sin(t) * np.sin(t)]), t5(t)])
xi_6 = lambda t: np.concatenate([np.array([2 * t ** 2, 3 * np.sin(6 * t) * np.cos(t), 5 * t * np.cos(t) * np.cos(t)]), t6(t)])
xi_7 = lambda t: np.concatenate([np.array([-1, 6 * np.sin(3 * t), 3 * t * np.sin(t)]), t7(t)])
xi_8 = lambda t: np.concatenate([np.array([t ** 2, 4 * np.cos(3 * t), t * np.sin(t)]), t8(t)])
xi_9 = lambda t: np.concatenate([np.array([3 * t, 5 * np.sin(2 * t), 2 * t * np.cos(t)]), t9(t)])

xi_lst = [xi_1, xi_2, xi_3, xi_4, xi_5, xi_6, xi_7, xi_8, xi_9]

In [ ]:
t_eval = np.linspace(0, 1, 100)
g0 = np.eye(4).ravel()

g_dot_lst = []
g_t_lst = []

for i in range(9): 
    g_dot = create_g_dot(xi_lst[i])
    g_t = solve_ivp_rk4(g_dot, g0, t_eval)
    g_t = g_t.reshape(len(t_eval), 4, 4)

    g_dot_lst.append(g_dot)
    g_t_lst.append(g_t)

for i, g_t in enumerate(g_t_lst):
    for g in g_t:
        assert is_SE3(g), f"Solution {i} is not in SO(3)"

In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt
# import pandas as pd
# from mpl_toolkits.mplot3d import Axes3D

# for i, g_t in enumerate(g_t_lst):
#     translations = g_t[:, :3, 3]

#     SE3_translation = pd.DataFrame({'x' : translations[:, 0], 'y' : translations[:, 1], 'z' : translations[:, 2]})
#     SE3_translation.to_csv(f"figures/syntetic_data/SE3_t_{i+1}.csv", index=False)

#     # Plotting the translations (trajectory)
#     fig = plt.figure()
#     ax = fig.add_subplot(111, projection='3d')

#     # Plot the trajectory
#     ax.plot(translations[:, 0], translations[:, 1], translations[:, 2], label='Translation Trajectory')
#     ax.scatter(translations[:, 0], translations[:, 1], translations[:, 2], c='red')  # Mark points for clarity

#     # Label and show the plot
#     ax.set_xlabel('X')
#     ax.set_ylabel('Y')
#     ax.set_zlabel('Z')
#     ax.set_title('3D Translation Trajectory')
#     ax.legend()
#     plt.show()

In [ ]:
def is_diff_plus(c): 
    if c[0] != 0: 
        print('c[0] != 0')
        return False
    if c[-1] != 1: 
        print(f'c[-1] != 1')
        print(c[-1])
        return False
    if np.diff(c).min() < 0: 
        print('np.diff(c).min() < 0')
        return False
    return True

def basis_function(n,x): 
    return np.sin(n*np.pi*x) / (n * np.pi)

def I(x): 
    return x 

def varphi_func(x, I, f, *args):
    return I(x) + f(x, *args)

def pi(w, epsilon):
    norm_w = np.linalg.norm(w, 1)
    # if norm_w > 1 - epsilon:
    #     print(f"norm_w: {norm_w}")
    scaling_factor = (1 - epsilon) / max(1 - epsilon, norm_w)
    return scaling_factor * w

def generate_and_transform_weights(random, epsilon, M, std):
    if random: weights = np.random.normal(0, std, M - 1)
    else: weights = np.ones(M - 1)
    weights = pi(weights, epsilon)
    return weights

def generate_delta_from_basis(x, M, random=True, epsilon=1e-8, std = 1):
    weights = generate_and_transform_weights(random, epsilon, M, std)
    delta = sum(weights[j - 1] * basis_function(j, x) for j in range(1, M))
    delta[np.abs(delta) < 1e-15] = 0
    return delta

In [ ]:
import matplotlib.pyplot as plt


M = 4
random = True
epsilon = 1e-8
std = 2
x = np.linspace(0, 1, 100)

np.random.seed(2)
varphi_1 = varphi_func(x, I, generate_delta_from_basis, M, random, epsilon, std)

np.random.seed(3)
varphi_2 = varphi_func(x, I, generate_delta_from_basis, M, random, epsilon, std)

np.random.seed(5)
varphi_3 = varphi_func(x, I, generate_delta_from_basis, M, random, epsilon, std)

plt.plot(x, varphi_1, label=r'$\varphi_1(x)$')
plt.plot(x, varphi_2, label=r'$\varphi_2(x)$')
plt.plot(x, varphi_3, label=r'$\varphi_3(x)$')

plt.legend()
plt.show()

In [ ]:
from SE3.utils.utils import * 

def SE3_move_to_origin(c):
    c_inv = SE3_inv(c[0])
    for i in range(c.shape[0]):
        c[i] = SE3_dot(c_inv, c[i])
    return c

def SE3_right_log_derivative_curve(I, c):
    dc = np.zeros((I.shape[0] - 1, 4, 4))
    for index in range(I.shape[0] - 1):
        dc[index] = SE3_log(SE3_dot(c[index + 1], SE3_inv(c[index]))) / (I[index + 1] - I[index])
    return dc

def SRVT(I, c):
    derivative = SE3_right_log_derivative_curve(I, c)
    norm_derivative = np.linalg.norm(derivative, axis=(1, 2))
    norm_derivative[norm_derivative < np.finfo(float).eps] = 1.0
    norm_derivative = np.sqrt(norm_derivative)
    return np.divide(derivative, norm_derivative[:, None, None])

def SE3_vee_curve(c):
    if len(c.shape) == 3:
        return np.array([SE3_vee_curve(c_i) for c_i in c])
    return SE3_vee(c)

In [ ]:
from SE3.utils.utils import *
from SO3.utils.reparameterization_utils import *

def find_reparameterization(c1, c2, depth = 10): 
    I = np.linspace(0, 1, len(c1))
    c1 = SE3_move_to_origin(c1)
    c2 = SE3_move_to_origin(c2)

    q1 = SE3_vee_curve(SRVT(I,c1))
    q2 = SE3_vee_curve(SRVT(I,c2))

    I_new = find_optimal_diffeomorphism_SE3(q1, q2, I , I, depth)
    return I_new

In [ ]:
# x = np.linspace(0, 1, 100)

# # Initialize empty dataframes
# df_varphi_1 = pd.DataFrame({'x' : x})
# df_varphi_2 = pd.DataFrame({'x' : x})
# df_varphi_3 = pd.DataFrame({'x' : x})

# from tqdm import tqdm


# pbar = tqdm(total=3 * 3)

# for i, g_func in enumerate(g_dot_lst[:3]):
#     for j, varphi in enumerate([varphi_1, varphi_2, varphi_3]):
#         c2 = solve_ivp_rk4(g_func, g0, x)
#         c2 = c2.reshape(len(x), 4, 4)

#         c1 = solve_ivp_rk4(g_func, g0, varphi)
#         c1 = c1.reshape(len(varphi), 4, 4)

#         I_new = find_reparameterization(c1, c2, depth = 10)

#         # Store the result in the appropriate dataframe
#         if j == 0:
#             df_varphi_1[f"g{i}"] = I_new
#         elif j == 1:
#             df_varphi_2[f"g{i}"] = I_new
#         else:
#             df_varphi_3[f"g{i}"] = I_new

#         pbar.update(1)

# pbar.close()

# df_varphi_1.to_csv('figures/syntetic_data/reparameterization_SE3/df_varphi_1.csv', index=False)
# df_varphi_2.to_csv('figures/syntetic_data/reparameterization_SE3/df_varphi_2.csv', index=False)
# df_varphi_3.to_csv('figures/syntetic_data/reparameterization_SE3/df_varphi_3.csv', index=False)

In [ ]:
# import numpy as np
# import pandas as pd
# import time 
# import os

# x = np.linspace(0, 1, 100)
# time_data = {}

# for i, g_func in enumerate(g_dot_lst[:3]):
#     for j, varphi in enumerate([varphi_1, varphi_2, varphi_3]):
#         # Prepare an empty list to store individual rows as dictionaries
#         data = []

#         for depth in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]:
#             start_time = time.time()

#             c2 = solve_ivp_rk4(g_func, g0, x)
#             c2 = c2.reshape(len(x), 4, 4)

#             c1 = solve_ivp_rk4(g_func, g0, varphi)
#             c1 = c1.reshape(len(varphi), 4, 4)

#             I_new = find_reparameterization(c1, c2, depth=depth)
#             I = np.linspace(0, 1, len(c1))

#             c2 = SE3_reparameterize(I_new, I, c2)
#             c1 = SE3_move_to_origin(c1)
#             c2 = SE3_move_to_origin(c2)

#             q1 = SE3_vee_curve(SRVT(I, c1))
#             q2 = SE3_vee_curve(SRVT(I, c2))

#             L2_distance = L2_metric(q1, q2, I, I)

#             elapsed_time = time.time() - start_time 

#             # Append the result as a dictionary to the data list
#             data.append({'depth': depth, 'L2_distance': L2_distance})

#             if depth not in time_data:
#                 time_data[depth] = []

#             time_data[depth].append(elapsed_time)

#         df = pd.DataFrame(data)
#         dir_path = 'figures/syntetic_data/reparameterization_SE3/depth_error'
#         os.makedirs(dir_path, exist_ok=True)
#         df.to_csv(f'{dir_path}/g{i}_varphi{j}.csv', index=False)

# mean_time_data = {}

# for depth, times in time_data.items():
#     mean_time_data[depth] = np.mean(times)

# # If you want to convert it to a DataFrame
# mean_time_df = pd.DataFrame(list(mean_time_data.items()), columns=['depth', 'mean_time'])

# # mean_time_df.to_csv(f'{dir_path}/mean_time.csv', index=False)



In [ ]:
import numpy as np
import pandas as pd
import time 

x = np.linspace(0, 1, 100)
time_data = {}
curves = {}

counter = 0
for i, g_func in enumerate(g_dot_lst[:3]):
    for j, varphi in enumerate([varphi_1, varphi_2, varphi_3, x]):
        curve = solve_ivp_rk4(g_func, g0, varphi)
        curve = curve.reshape(len(varphi), 4, 4)
        curves[f"c_{counter}"] = curve
        counter += 1


In [ ]:
# distances = np.zeros((len(curves), len(curves)))

# total = (len(curves) * len(curves) - len(curves)) // 2
# pbar = tqdm(total=total)

# for i, c1 in enumerate(curves.values()):
#     for j, c2 in enumerate(curves.values()):
#         if i <= j:
#             continue
            
#         I_new = find_reparameterization(c1, c2, depth=4)
#         I = np.linspace(0, 1, len(c1))

#         c2 = SE3_reparameterize(I_new, I, c2)
#         c1 = SE3_move_to_origin(c1)
#         c2 = SE3_move_to_origin(c2)

#         q1 = SE3_vee_curve(SRVT(I, c1))
#         q2 = SE3_vee_curve(SRVT(I, c2))

#         L2_distance = L2_metric(q1, q2, I, I)
#         distances[i, j] = L2_distance
#         distances[j, i] = L2_distance

#         pbar.update(1)

# pbar.close()

In [ ]:
# import matplotlib.pyplot as plt
# from tikzplotlib import save as tikz_save

# # Plot the distance matrix
# plt.figure(figsize=(8, 5))
# plt.imshow(distances, cmap='hot', interpolation='nearest')
# plt.colorbar()
# # plt.show()

# plt.savefig(f"figures/syntetic_data/distance_matrix/SE3.png")

### Signature

In [ ]:
# import iisignature
# import numpy as np
# from scipy.stats import wasserstein_distance
# import matplotlib.pyplot as plt
# import iisignature
# import numpy as np
# from scipy.stats import wasserstein_distance
# from sklearn.decomposition import PCA
# from sklearn.preprocessing import StandardScaler
# from scipy.spatial.distance import pdist, squareform


# for level in [6,7]:
#     preprocessed_data = iisignature.prepare(6, level)
#     signatures = {}

#     for name, c in curves.items():
#         I = np.linspace(0, 1, len(c))
#         c = SE3_move_to_origin(c)
#         q = SE3_vee_curve(SE3_right_log_derivative_curve(I, c))
#         signatures[name] = iisignature.logsig(q, preprocessed_data)

#     signature_matrix = np.zeros((len(signatures), len(signatures)))
#     for i, s1 in enumerate(signatures.values()):
#         for j, s2 in enumerate(signatures.values()):
#             # distance =  np.linalg.norm(s1 / np.linalg.norm(s1) - s2 / np.linalg.norm(s2))
#             # distance = np.linalg.norm(s1 - s2)
#             distance = 1 - np.dot(s1, s2) / (np.linalg.norm(s1) * np.linalg.norm(s2))
#             # distance = wasserstein_distance(s1 / np.linalg.norm(s1), s2 / np.linalg.norm(s2))
#             signature_matrix[i, j] = distance


#     plt.figure(figsize=(8, 5))
#     plt.imshow(signature_matrix, cmap='viridis', interpolation='nearest')
#     plt.colorbar()
#     # plt.savefig(f"figures/syntetic_data/distance_matrix/SE3_signature_{level}.png", bbox_inches='tight', pad_inches=0)
#     plt.show()

#     # Standardize the data
#     scaler = StandardScaler()
#     X_standardized = scaler.fit_transform(signature_matrix)

#     # Apply PCA
#     num_components = 2
#     pca = PCA(n_components=num_components)
#     X_reduced = pca.fit_transform(X_standardized)

#     # Compute the pairwise distance matrix
#     distance_matrix = squareform(pdist(X_reduced, metric='cosine'))

#     # Visualize the distance matrix
#     plt.figure(figsize=(8, 5))
#     plt.imshow(distance_matrix, cmap='viridis', interpolation='nearest')
#     plt.colorbar()
#     plt.show()

### SE3(3)^n 

In [ ]:
curves = {}

for i, varphi in enumerate([varphi_1, varphi_2, varphi_3, x]):
    curve = np.zeros((len(x), 9, 4, 4))
    for j, g_func in enumerate(g_dot_lst):
        c_j = solve_ivp_rk4(g_func, g0, varphi)
        c_j = c_j.reshape(len(varphi), 4, 4)
        curve[:, j] = c_j

    # Reshape curve from (100, 3, 3, 3) to (3, 100, 3, 3)
    curve = np.moveaxis(curve, 1, 0)
    curves[f"c_1_{i+1}"] = curve[0:3]
    curves[f"c_2_{i+1}"] = curve[3:6]
    curves[f"c_3_{i+1}"] = curve[6:9]

keys = sorted(curves.keys(), key=lambda x: (int(x.split('_')[1]), int(x.split('_')[2])))
sorted_curves = {key: curves[key] for key in keys}

In [ ]:
print(sorted_curves.keys())

In [ ]:
def skew_matrix_to_vector_several_rotations(c):
    vector = np.zeros((c.shape[1], 6 * c.shape[0]))
    for i in range(c.shape[1]):
        vector[i] =  SE3_vee_curve(c[:, i]).flatten()
    return vector

def move_several_rotations_origins_to_identity(c) -> np.ndarray:
    new_rotation = np.zeros(c.shape)
    for i, joint in enumerate(c):
        new_rotation[i] = SE3_move_to_origin(joint)
    return new_rotation

def SRVT_multiple_rotations(I: np.ndarray, movement: np.ndarray) -> np.ndarray:
    SRVT_ = np.zeros((movement.shape[0], movement.shape[1] - 1, 4, 4))
    for i, rotation in enumerate(movement):
        SRVT_[i] = SRVT(I, rotation)
    return SRVT_

def find_reparameterization_several(c1, c2, depth = 10): 
    I = np.linspace(0, 1, c1.shape[1])

    c1 = move_several_rotations_origins_to_identity(c1)
    c2 = move_several_rotations_origins_to_identity(c2)
    SRVT_1 = SRVT_multiple_rotations(I, c1)
    SRVT_2 = SRVT_multiple_rotations(I, c2)
    q1 = skew_matrix_to_vector_several_rotations(SRVT_1)
    q2 = skew_matrix_to_vector_several_rotations(SRVT_2)
    I_new = find_optimal_diffeomorphism(q1, q2, I , I, depth)
    return I_new

def reparameterize_multiple_rotations(I_new, I, c):
    """
    Creates the new movement
    Input:
        I_new: new parameterization
        I: old parameterization
        c: old movement
    Output:
        c_new: new movement
    """
    c_new = np.zeros(c.shape)
    for i in range(c.shape[0]):
        c_new[i] = SE3_reparameterize(I_new, I, c[i])
    return c_new

# find_reparameterization_several(curves['c_0'], curves['c_1'], depth=1)

In [ ]:
from SO3.movement_data.calculate_reparameterized_distance import reparameterized_distance

# c0 = curves['c_0']
# c1 = curves['c_1']
# c2 = curves['c_2']
# c_identity = curves['c_3']

# depth = 10
# I_0 = find_reparameterization_several(c0, c_identity, depth=depth)
# I_1 = find_reparameterization_several(c1, c_identity, depth=depth)
# I_2 = find_reparameterization_several(c2, c_identity, depth=depth)

In [ ]:
# df_varphi_1 = pd.DataFrame({'x' : x})
# df_varphi_2 = pd.DataFrame({'x' : x})
# df_varphi_3 = pd.DataFrame({'x' : x})

# depth = 10

# from tqdm import tqdm
# pbar = tqdm(total=3 * 3)

# for i, key in enumerate(['c_1_1', 'c_1_2', 'c_1_3']):
#     c1 = curves[key]
#     c2 = curves['c_1_4']
#     I_new = find_reparameterization_several(c1, c2, depth=depth)
#     df_varphi_1[f"g{i}"] = I_new
#     pbar.update(1)

# for i, key in enumerate(['c_2_1', 'c_2_2', 'c_2_3']):
#     c1 = curves[key]
#     c2 = curves['c_2_4']
#     I_new = find_reparameterization_several(c1, c2, depth=depth)
#     df_varphi_2[f"g{i}"] = I_new
#     pbar.update(1)

# for i, key in enumerate(['c_3_1', 'c_3_2', 'c_3_3']):
#     c1 = curves[key]
#     c2 = curves['c_3_4']
#     I_new = find_reparameterization_several(c1, c2, depth=depth)
#     df_varphi_3[f"g{i}"] = I_new
#     pbar.update(1)

# pbar.close()


# df_varphi_1.to_csv('figures/syntetic_data/reparameterization_SE3_3/df_varphi_1.csv', index=False)
# df_varphi_2.to_csv('figures/syntetic_data/reparameterization_SE3_3/df_varphi_2.csv', index=False)
# df_varphi_3.to_csv('figures/syntetic_data/reparameterization_SE3_3/df_varphi_3.csv', index=False)

In [ ]:
# plt.plot(x, I_0, label=r'$\hat \varphi_1$', color='blue', marker = '.')
# plt.plot(x, varphi_1, label=r'$\varphi_1$', color='blue')

# plt.plot(x, I_1, label=r'$\hat \varphi_2$', color='red', marker = '.')
# plt.plot(x, varphi_2, label=r'$\varphi_2$', color='red')

# plt.plot(x, I_2, label=r'$\hat \varphi_3$', color='green', marker = '.')
# plt.plot(x, varphi_3, label=r'$\varphi_3$', color='green')

# plt.legend()
# plt.show()

In [ ]:
# import numpy as np
# import pandas as pd
# import time 
# import os
# from tqdm import tqdm

# x = np.linspace(0, 1, 100)
# time_data = {}

# pbar = tqdm(total=9)

# for key in sorted_curves.keys():
#     _, i, j = key.split('_')

#     # If equidistant parameterization, skip
#     if j == '4':
#         continue
    
#     c1 = curves[key]
#     c2 = curves[f'c_{i}_4']

#     # Prepare an empty list to store individual rows as dictionaries
#     data = []

#     for depth in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]:
#         start_time = time.time()

#         I_new = find_reparameterization_several(c1, c2, depth=depth)
#         c2 = reparameterize_multiple_rotations(I_new, x, c2)

#         c1 = move_several_rotations_origins_to_identity(c1)
#         c2 = move_several_rotations_origins_to_identity(c2)

#         SRVT_1 = SRVT_multiple_rotations(x, c1)
#         SRVT_2 = SRVT_multiple_rotations(x, c2)

#         q1 = skew_matrix_to_vector_several_rotations(SRVT_1)
#         q2 = skew_matrix_to_vector_several_rotations(SRVT_2)

#         L2_distance = L2_metric(q1, q2, x, x)

#         elapsed_time = time.time() - start_time 

#         # Append the result as a dictionary to the data list
#         data.append({'depth': depth, 'L2_distance': L2_distance})

#         if depth not in time_data:
#             time_data[depth] = []

#         time_data[depth].append(elapsed_time)

#         df = pd.DataFrame(data)
#         dir_path = 'figures/syntetic_data/reparameterization_SE3_3/depth_error'
#         os.makedirs(dir_path, exist_ok=True)
#         df.to_csv(f'{dir_path}/g{int(i)-1}_varphi{int(j)-1}.csv', index=False)
    
#     print(f"Done with {key}")
#     pbar.update()

# pbar.close()

# mean_time_data = {}

# for depth, times in time_data.items():
#     mean_time_data[depth] = np.mean(times)

# # If you want to convert it to a DataFrame
# mean_time_df = pd.DataFrame(list(mean_time_data.items()), columns=['depth', 'mean_time'])

# mean_time_df.to_csv(f'{dir_path}/mean_time.csv', index=False)

In [ ]:
# from tqdm import tqdm

# depth = 10
# distance_matrix = np.zeros((len(sorted_curves), len(sorted_curves)))
# total = (len(sorted_curves) * len(sorted_curves) - len(sorted_curves)) // 2

# pbar = tqdm(total=total)

# for i, c1 in enumerate(sorted_curves.values()):
#     for j, c2 in enumerate(sorted_curves.values()):
#         if i <= j:
#             continue

#         I_new = find_reparameterization_several(c1, c2, depth=depth)
#         c2 = reparameterize_multiple_rotations(I_new, x, c2)

#         c1 = move_several_rotations_origins_to_identity(c1)
#         c2 = move_several_rotations_origins_to_identity(c2)

#         SRVT_1 = SRVT_multiple_rotations(x, c1)
#         SRVT_2 = SRVT_multiple_rotations(x, c2)

#         q1 = skew_matrix_to_vector_several_rotations(SRVT_1)
#         q2 = skew_matrix_to_vector_several_rotations(SRVT_2)

#         distance = L2_metric(q1, q2, x, x)

#         distance_matrix[i, j] = distance
#         distance_matrix[j, i] = distance

#         pbar.update()

# pbar.close()

# plt.figure(figsize=(8, 5))
# plt.imshow(distance_matrix, cmap='hot', interpolation='nearest')
# plt.colorbar()
# # plt.savefig(f"figures/syntetic_data/distance_matrix/SE3_3.png", bbox_inches='tight', pad_inches=0)
# plt.show()

In [ ]:
# import iisignature
# import numpy as np

# def create_parameterization_several_rotations(movement: np.ndarray) -> np.ndarray:
#     return np.linspace(0, 1, movement.shape[1])    

# def right_log_several_rotations(I: np.ndarray, movement: np.ndarray): 
#     right_log = np.zeros((movement.shape[0], movement.shape[1] - 1, 4, 4))
#     for i, rotation in enumerate(movement):
#         right_log[i] = SE3_right_log_derivative_curve(I, rotation)
#     return right_log


# for level in [1, 2, 3, 4, 5]:
#     preprocessed_data = iisignature.prepare(18, level)
#     signatures = {}

#     for key, c in sorted_curves.items():
#         c = move_several_rotations_origins_to_identity(c)
#         I = create_parameterization_several_rotations(c)
#         right_log = right_log_several_rotations(I, c)
#         vector = skew_matrix_to_vector_several_rotations(right_log)
#         signatures[key] = iisignature.logsig(vector, preprocessed_data)

#     print(f"Done with level {level}")

#     signature_matrix = np.zeros((len(signatures), len(signatures)))
#     for i, s1 in enumerate(signatures.values()):
#         for j, s2 in enumerate(signatures.values()):
#             distance = 1 - np.dot(s1, s2) / (np.linalg.norm(s1) * np.linalg.norm(s2))
#             signature_matrix[i, j] = distance

#     plt.figure(figsize=(8, 5))
#     plt.imshow(signature_matrix, cmap='hot', interpolation='nearest')
#     plt.colorbar()
#     plt.savefig(f"figures/syntetic_data/distance_matrix/SE3_3_signature_{level}.png", bbox_inches='tight', pad_inches=0)
#     # plt.show()

### Perturbation

In [ ]:
t_eq = np.linspace(0, 1, 100)

def pertubate_parameterization(t, max_pertubation):
    t = t.copy()
    perturbation = np.random.normal(0, max_pertubation, len(t) - 2)
    # perturbation = np.random.uniform(-max_pertubation, max_pertubation, len(t) - 2)
    perturbation = np.clip(perturbation, -max_pertubation, max_pertubation)
    t[1:-1] += perturbation
    return t

In [ ]:
# from tqdm import tqdm


# n = 10
# num_g = 3

# pbar = tqdm(total=num_g)
# distances = np.zeros((num_g, n))


# for i, g_dot in enumerate(g_dot_lst[:num_g]):

#     perturbations = []
#     max_pertubation = (t_eq[1] - t_eq[0]) / 2 - 1e-12

#     c_ref = solve_ivp_rk4(g_dot_lst[0], g0, t_eq)
#     c_ref = c_ref.reshape(len(varphi), 4, 4)

#     c1 = SE3_move_to_origin(c_ref)
#     q1 = SE3_vee_curve(SRVT(t_eq,c1))



#     for j in range(n):
#         t_pertubated = pertubate_parameterization(t_eq, max_pertubation)
#         c_pertubated = solve_ivp_rk4(g_dot_lst[0], g0, t_pertubated)
#         c_pertubated = c_pertubated.reshape(len(varphi), 4, 4)

#         c2 = SE3_move_to_origin(c_pertubated)
#         q2 = SE3_vee_curve(SRVT(t_eq,c2))

#         distances[i, j] = L2_metric_SE3(q1, q2, t_eq, t_eq)
#         perturbations.append(max_pertubation)
        
#         max_pertubation = max_pertubation / 2
        
#     pbar.update(1)
    
# pbar.close()


In [ ]:
# # Plot the distances with labels
# for i, distance in enumerate(distances):
#     plt.loglog(perturbations, distance, label=f'$\omega_{i}$')

# # Plot the reference line for order 1
# x = np.linspace(min(perturbations), max(perturbations), 100)
# plt.loglog(x, x**1, label=r'$\mathcal{O}(\epsilon^{1})$', linestyle='--', color='black')


# # Improve plot aesthetics
# plt.xlabel(r'$\epsilon$')
# plt.ylabel(r'$\|q - q \circ \varphi_{\epsilon}\|_{L^2}$')
# plt.legend(fontsize='small')
# plt.title('Perturbation Analysis')
# # plt.grid(True, which="both", ls="--")
# plt.tight_layout()

# # Show the plot
# plt.show()


In [ ]:
# df = pd.DataFrame(distances)
# df = df.T
# df.columns = ['c1', 'c2', 'c3']
# df['perturbation'] = perturbations

# df.to_csv('figures/syntetic_data/perturbation-analysis/SE3.csv', index=False)

In [ ]:
# from tqdm import tqdm


# n = 10
# num_g = 3

# pbar = tqdm(total=num_g)
# distances = np.zeros((num_g, n))


# for i, g_dot in enumerate(g_dot_lst[:num_g]):

#     perturbations = []
#     max_pertubation = (t_eq[1] - t_eq[0]) / 2 - 1e-12

#     c_ref = solve_ivp_rk4(g_dot_lst[0], g0, t_eq)
#     c_ref = c_ref.reshape(len(varphi), 4, 4)

#     c1 = SE3_move_to_origin(c_ref)
#     q1 = SE3_vee_curve(SRVT(t_eq,c1))



#     for j in range(n):
#         t_pertubated = pertubate_parameterization(t_eq, max_pertubation)
#         c_pertubated = solve_ivp_rk4(g_dot_lst[0], g0, t_pertubated)
#         c_pertubated = c_pertubated.reshape(len(varphi), 4, 4)
#         c2 = SE3_move_to_origin(c_pertubated)


#         I_new = find_reparameterization(c1, c2, depth=10)
#         # I = np.linspace(0, 1, len(c1))
#         c2 = SE3_reparameterize(I_new, t_eq, c2)

#         c2 = SE3_move_to_origin(c2)
#         q2 = SE3_vee_curve(SRVT(t_eq,c2))

#         distances[i, j] = L2_metric_SE3(q1, q2, t_eq, t_eq)
#         perturbations.append(max_pertubation)
        
#         max_pertubation = max_pertubation / 2
        
#     pbar.update(1)
    
# pbar.close()


In [ ]:
# df = pd.DataFrame(distances)
# df = df.T
# df.columns = ['c1', 'c2', 'c3']
# df['perturbation'] = perturbations

# df.to_csv('figures/syntetic_data/perturbation-analysis/reparameterized-SE3.csv', index=False)

### Signature

In [ ]:
from tqdm import tqdm
import iisignature

level = 8
preprocessed_data = iisignature.prepare(6, level)

n = 10
num_g = 3

pbar = tqdm(total=num_g)
distances = np.zeros((num_g, n))


for i, g_dot in enumerate(g_dot_lst[:num_g]):

    perturbations = []
    max_pertubation = (t_eq[1] - t_eq[0]) / 2 - 1e-12

    c_ref = solve_ivp_rk4(g_dot, g0, t_eq)
    c_ref = c_ref.reshape(len(varphi), 4, 4)

    c1 = SE3_move_to_origin(c_ref)
    q1 = SE3_vee_curve(SE3_right_log_derivative_curve(t_eq, c1))

    for j in range(n):

        t_pertubated = pertubate_parameterization(t_eq, max_pertubation)
        c_pertubated = solve_ivp_rk4(g_dot, g0, t_pertubated)
        c_pertubated = c_pertubated.reshape(len(varphi), 4, 4)

        c2 = SE3_move_to_origin(c_pertubated)
        q2 = SE3_vee_curve(SE3_right_log_derivative_curve(t_eq, c2))

        sig1 = iisignature.logsig(q1, preprocessed_data)
        sig2 = iisignature.logsig(q2, preprocessed_data)

        distances[i, j] = np.linalg.norm(sig1 / np.linalg.norm(sig1) - sig2 / np.linalg.norm(sig2))
        perturbations.append(max_pertubation)
        
        max_pertubation = max_pertubation / 2
        
    pbar.update(1)
    
pbar.close()

In [ ]:
# Plot the distances with labels
for i, distance in enumerate(distances):
    plt.loglog(perturbations, distance, label=f'$\omega_{i}$')

# Plot the reference line for order 1
x = np.linspace(min(perturbations), max(perturbations), 100)
plt.loglog(x, x**1, label=r'$\mathcal{O}(\epsilon^{1})$', linestyle='--', color='black')


# Improve plot aesthetics
plt.xlabel(r'$\epsilon$')
plt.ylabel(r'$\|q - q \circ \varphi_{\epsilon}\|_{L^2}$')
plt.legend(fontsize='small')
plt.title('Perturbation Analysis')
# plt.grid(True, which="both", ls="--")
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
df = pd.DataFrame(distances)
df = df.T
df.columns = ['c1', 'c2', 'c3']
df['perturbation'] = perturbations
df
# df.to_csv('figures/syntetic_data/perturbation-analysis/signature-SE3.csv', index=False)

### Gradient Based

In [ ]:
import numpy as np
from scipy.optimize import minimize
from scipy.spatial.transform import Rotation
from joblib import Parallel, delayed

def find_optimal_nodes(x_init, cost, niter=10, n_jobs=1):
    n = len(x_init)
    bounds = [(0, 0)] + [(None, None)] * (n - 2) + [(1, 1)]

    def increasing_constraint(x, i):
        return x[i + 1] - x[i]

    constraints = [{'type': 'ineq', 'fun': increasing_constraint, 'args': (i,)} for i in range(n - 1)]

    options = {
        'disp': False,
        'ftol': 1e-9,
        # 'maxiter': 20,
        'maxiter': 1, 
        # 'eps': 1.4901161193847656e-08
    }

    def run_optimization(i):
        x_init = np.sort(np.random.uniform(0, 1, 100))
        x_init[0], x_init[-1] = 0, 1
        
        result = minimize(cost,
                          x_init,
                          method='SLSQP',
                          bounds=bounds,
                          constraints=constraints,
                          options=options)
        return result

    results = Parallel(n_jobs=n_jobs)(delayed(run_optimization)(i) for i in range(niter))

    return min(results, key=lambda result: result.fun).x

def se3_difference(T1, T2):
    assert T1.shape == T2.shape, 'T1 and T2 must have the same shape'
    diffs = np.array([log_map(np.dot(np.linalg.inv(T1[i]), T2[i])) for i in range(T1.shape[0])])
    norm_sq = np.mean(np.linalg.norm(diffs, axis=(1, 2))**2)
    return norm_sq

def create_se3_slerp_f(x, g):
    # Calculate the log map of g[i-1].T @ g[i] for each pair of consecutive elements in g
    xi_hat = [SE3_log(np.linalg.inv(g[i-1]) @ g[i]) for i in range(1, len(g))]

    def interpSE3(x_eval):
        result = []
        for x_e in x_eval:
            # Find where x_eval comes in x
            index = np.searchsorted(x, x_e, side='right')
            # Handle out of bounds by clamping to valid range
            index = np.clip(index, 1, len(x) - 1)
            # Interpolation calculation
            alpha = (x_e - x[index-1]) / (x[index] - x[index-1])
            # Interpolate in the Lie algebra and exponentiate
            interpolated_matrix = g[index-1] @ SE3_exp(alpha * xi_hat[index-1])
            result.append(interpolated_matrix)
        
        return np.array(result)
    
    return interpSE3

def C_factory(f1, y2):
    def C(x):
        x_eq = np.linspace(0, 1, len(x))
        x = np.clip(x, 0, 1)
        y1 = create_se3_slerp_f(x_eq, f1(x))(x_eq)
        return se3_difference(y1, y2)
    return C

curves = {}
counter = 0
for i, g_func in enumerate(g_dot_lst[:3]):
    for j, varphi in enumerate([varphi_1, varphi_2, varphi_3, x]):
        curve = solve_ivp_rk4(g_func, g0, varphi)
        curve = curve.reshape(len(varphi), 4, 4)
        curves[f"c_{counter}"] = curve
        counter += 1


In [ ]:
# y2 = curves['c_6']
# y1 = curves['c_7']
# x = np.linspace(0, 1, len(y1))

In [ ]:
import os
num_cores = os.cpu_count()
print(f'Number of CPU cores available: {num_cores}')

def find_reparameterization_SLERP(x, c_fit, c_target, restart, num_cores):
    f_fit = create_se3_slerp_f(x, c_fit)
    f_target = create_se3_slerp_f(x, c_target)
    y_target = f_target(x)
    cost = C_factory(f_fit, y_target)
    x_opt = find_optimal_nodes(x, cost, restart, num_cores)
    return x_opt

# x_opt = find_reparameterization_SLERP(x, c_fit = y1, c_target = y2, restart = 5, num_cores = num_cores)

In [ ]:
# plt.plot(x, x_opt, marker='.')
# plt.plot(x, varphi_3)
# plt.show()

In [ ]:
import pandas as pd

x = np.linspace(0, 1, 100)

# Initialize empty dataframes
df_varphi_1 = pd.DataFrame({'x' : x})
df_varphi_2 = pd.DataFrame({'x' : x})
df_varphi_3 = pd.DataFrame({'x' : x})

varphi = varphi_1
for i, g_func in enumerate(g_dot_lst[:3]):
    c_fit = solve_ivp_rk4(g_func, g0, x)
    c_fit = c_fit.reshape(len(x), 4, 4)

    c_target = solve_ivp_rk4(g_func, g0, varphi)
    c_target = c_target.reshape(len(varphi), 4, 4)

    x_opt = find_reparameterization_SLERP(x, c_fit = c_fit, c_target = c_target, restart = 5, num_cores = num_cores)
    df_varphi_1[f"g{i}"] = x_opt

    plt.plot(x, varphi)
    plt.plot(x, x_opt, marker='.')
    plt.show()

# df_varphi_1.to_csv('figures/syntetic_data/reparameterization_SE3_SLERP/df_varphi_1.csv', index=False)

print("Done with varphi_1")

varphi = varphi_2
for i, g_func in enumerate(g_dot_lst[:3]):
    c_fit = solve_ivp_rk4(g_func, g0, x)
    c_fit = c_fit.reshape(len(x), 4, 4)

    c_target = solve_ivp_rk4(g_func, g0, varphi)
    c_target = c_target.reshape(len(varphi), 4, 4)

    x_opt = find_reparameterization_SLERP(x, c_fit = c_fit, c_target = c_target, restart = 5, num_cores = num_cores)
    df_varphi_2[f"g{i}"] = x_opt

    plt.plot(x, varphi)
    plt.plot(x, x_opt, marker='.')
    plt.show()

# df_varphi_2.to_csv('figures/syntetic_data/reparameterization_SE3_SLERP/df_varphi_2.csv', index=False)

print("Done with varphi_2")

varphi = varphi_3
for i, g_func in enumerate(g_dot_lst[:3]):
    c_fit = solve_ivp_rk4(g_func, g0, x)
    c_fit = c_fit.reshape(len(x), 4, 4)

    c_target = solve_ivp_rk4(g_func, g0, varphi)
    c_target = c_target.reshape(len(varphi), 4, 4)

    x_opt = find_reparameterization_SLERP(x, c_fit = c_fit, c_target = c_target, restart = 5, num_cores = num_cores)
    df_varphi_3[f"g{i}"] = x_opt

    plt.plot(x, varphi)
    plt.plot(x, x_opt, marker='.')
    plt.show()

# df_varphi_3.to_csv('figures/syntetic_data/reparameterization_SE3_SLERP/df_varphi_3.csv', index=False)

print("Done with varphi_3")

In [ ]:
curves = {}

for i, varphi in enumerate([varphi_1, varphi_2, varphi_3, x]):
    curve = np.zeros((len(x), 9, 4, 4))
    for j, g_func in enumerate(g_dot_lst):
        c_j = solve_ivp_rk4(g_func, g0, varphi)
        c_j = c_j.reshape(len(varphi), 4, 4)
        curve[:, j] = c_j

    # Reshape curve from (100, 3, 3, 3) to (3, 100, 3, 3)
    curve = np.moveaxis(curve, 1, 0)
    curves[f"c_1_{i+1}"] = curve[0:3]
    curves[f"c_2_{i+1}"] = curve[3:6]
    curves[f"c_3_{i+1}"] = curve[6:9]

keys = sorted(curves.keys(), key=lambda x: (int(x.split('_')[1]), int(x.split('_')[2])))
sorted_curves = {key: curves[key] for key in keys}

def rotation_difference_multiple(y1, y2):
    tot_diff = 0
    for i in range(len(y1)):
        diffs = np.array([log_map(np.dot(np.linalg.inv(y1[i][j]), y2[i][j])) for j in range(len(y1[i]))])
        tot_diff += np.mean(np.linalg.norm(diffs, axis=(1, 2))**2)
    return tot_diff

def C_factory_multiple(f1, y2):
    def C(x):
        x_eq = np.linspace(0, 1, len(x))
        x = np.clip(x, 0, 1)
        y1 = [create_se3_slerp_f(x_eq, f1_i(x))(x_eq) for f1_i in f1]
        return rotation_difference_multiple(y1, y2)
    return C

def find_optimal_nodes(x_init, cost, niter=10, n_jobs=1):
    n = len(x_init)
    bounds = [(0, 0)] + [(None, None)] * (n - 2) + [(1, 1)]

    def increasing_constraint(x, i):
        return x[i + 1] - x[i]

    constraints = [{'type': 'ineq', 'fun': increasing_constraint, 'args': (i,)} for i in range(n - 1)]

    options = {
        'disp': False,
        'ftol': 1e-9,
        'maxiter': 12,
        # 'eps': 1.4901161193847656e-08
    }

    def run_optimization(i):
        x_init = np.sort(np.random.uniform(0, 1, 100))
        x_init[0], x_init[-1] = 0, 1
        
        result = minimize(cost,
                          x_init,
                          method='SLSQP',
                          bounds=bounds,
                          constraints=constraints,
                          options=options)
        return result

    results = Parallel(n_jobs=n_jobs)(delayed(run_optimization)(i) for i in range(niter))

    return min(results, key=lambda result: result.fun).x

def find_reparameterization_SLERP_multiple(x, c_fit, c_target, restart, num_cores):
    f_fit = [create_se3_slerp_f(x, c) for c in c_fit]
    # f_target = [create_se3_slerp_f(x, c) for c in c_target]
    # y_target = [f(x) for f in f_target]
    cost = C_factory_multiple(f_fit, c_target)
    x_opt = find_optimal_nodes(x, cost, restart, num_cores)
    return x_opt

In [ ]:
df_varphi_1 = pd.DataFrame({'x' : x})
df_varphi_2 = pd.DataFrame({'x' : x})
df_varphi_3 = pd.DataFrame({'x' : x})

for i, key in enumerate(['c_1_1', 'c_1_2', 'c_1_3']):
    c_target = curves[key]
    c_fit = curves['c_1_4']
    x_opt = find_reparameterization_SLERP_multiple(x, c_fit, c_target, restart=5, num_cores=num_cores)
    df_varphi_1[f"g{i}"] = x_opt
        
    plt.plot(x, varphi_1)
    plt.plot(x, x_opt, marker='.')
    plt.show()

# df_varphi_1.to_csv('figures/syntetic_data/reparameterization_SE3_3_SLERP/df_varphi_1.csv', index=False)
print("Done with varphi_1")

for i, key in enumerate(['c_2_1', 'c_2_2', 'c_2_3']):
    c_target = curves[key]
    c_fit = curves['c_2_4']
    x_opt = find_reparameterization_SLERP_multiple(x, c_fit, c_target, restart=5, num_cores=num_cores)
    df_varphi_2[f"g{i}"] = x_opt

    plt.plot(x, varphi_2)
    plt.plot(x, x_opt, marker='.')
    plt.show()

# df_varphi_2.to_csv('figures/syntetic_data/reparameterization_SE3_3_SLERP/df_varphi_2.csv', index=False)
print("Done with varphi_2")

for i, key in enumerate(['c_3_1', 'c_3_2', 'c_3_3']):
    c_target = curves[key]
    c_fit = curves['c_3_4']

    x_opt = find_reparameterization_SLERP_multiple(x, c_fit, c_target, restart=5, num_cores=num_cores)
    df_varphi_3[f"g{i}"] = x_opt

    plt.plot(x, varphi_3)
    plt.plot(x, x_opt, marker='.')
    plt.show()

# df_varphi_3.to_csv('figures/syntetic_data/reparameterization_SE3_3_SLERP/df_varphi_3.csv', index=False)
print("Done with varphi_3")

In [ ]:
curves = {}
counter = 0
for i, g_func in enumerate(g_dot_lst[:3]):
    for j, varphi in enumerate([varphi_1, varphi_2, varphi_3, x]):
        curve = solve_ivp_rk4(g_func, g0, varphi)
        curve = curve.reshape(len(varphi), 4, 4)
        curves[f"c_{counter}"] = curve
        counter += 1

In [ ]:
total = (len(curves) * len(curves) - len(curves)) // 2
distance_matrix = np.zeros((len(curves), len(curves)))
counter = 0
x = np.linspace(0, 1, 100)
for i, c1 in enumerate(curves.values()):
    for j, c2 in enumerate(curves.values()):
        if i <= j:
            continue

        x_opt = find_reparameterization_SLERP(x, c1, c2, restart=5, num_cores=num_cores)
        f1 = create_se3_slerp_f(x, c1)
        x_opt = np.clip(x_opt, 0, 1)
        c1_new = create_se3_slerp_f(x, f1(x_opt))(x)
        distance = se3_difference(c1_new, c2)
        distance_matrix[i, j] = distance
        distance_matrix[j, i] = distance

        counter += 1
        print(f"Done : {counter} / {total}")

In [ ]:
plt.figure(figsize=(8, 5))
plt.imshow(distance_matrix, cmap='hot', interpolation='nearest')
plt.colorbar()
plt.show()
# plt.savefig(f"figures/syntetic_data/distance_matrix/SE3_interpolation.png", bbox_inches='tight', pad_inches=0)